In [1]:
#libreries
import pandas as pd
import numpy as np
#to convert the pivot matrix into a matrix
from scipy.sparse import csr_matrix
#NearsetNeighbors is a un-supervised learning. (is not knn)
from sklearn.neighbors import NearestNeighbors

In [2]:
class CollaborativeFiltering:
    def __init__(self, rating_df:str, item_df:str, userId:float, itemId:float,itemTitle:str, rating:float):
        '''Paths of the data sets'''
        #rating data set, that contains the userId, itemId and rating
        self.rating_df = rating_df
        #item data set, that contains the itemId and itemTitle
        self.item_df = item_df

        '''Columns of the data sets'''
        self.userId = userId
        self.itemId = itemId
        self.itemTitle = itemTitle
        self.rating = rating
        
    def import_data(self):
        '''Import data from csv files'''
        self.rating_df = pd.read_csv(self.rating_df, usecols= [self.userId, self.itemId, self.rating])
        self.item_df = pd.read_csv(self.item_df, usecols= [self.itemId, self.itemTitle])
        return self.rating_df, self.item_df
    
    def clean_data(self):
        '''clean data from csv files'''
        #Declaring global df, to use it in other methods
        global df
        #to merge the two data sets
        df = pd.merge(self.rating_df, self.item_df, on = self.itemId)
        #remove the rows with null values
        df. dropna(
            #axis = 0 means removes ROWS that contains NULL values
            axis= 0 , 
            #remove from column itemTitle
            subset= [self.itemTitle],)
        return df
    
    def ratingCount (self, perItem: bool = True, ofTable: bool = False):
        '''This function is to get some statistics from the data'''
        self.perItem = perItem
        self.ofTable = ofTable

        global item_ratingCount
        global df_ratingCount

        
        item_ratingCount= (df.
            #group by itemTitle and count the number of ratings
            groupby(by = [self.itemTitle])[self.rating].count().
            #reset the index
            reset_index().
            #rame the col as ratingCount
            rename(columns = {self.rating: 'ratingCount'})
            #show the title and total number of ratings
            [[self.itemTitle, 'ratingCount']]
                            )
            #return item_ratingCount.head()

        self.ofTable == True
        df_ratingCount = df.merge(
            item_ratingCount, 
            left_on= self.itemTitle,
            right_on= self.itemTitle,
            how = 'left')
            #return df_ratingCount.head() 

        if perItem == True and ofTable == False:
            return item_ratingCount.head()
        # perItem == True and ofTable == True:
        else:    
            return df_ratingCount.head()

    def statistics(self):
        '''This function is to get some statistics from the data'''
        return item_ratingCount['ratingCount'].describe()
    
    def create_user_item_matrix(self, popularity = 50):
        '''This function will be based only on popular items'''

        rating_popular_item = df_ratingCount.query('ratingCount >= @popularity')
        #return rating_popular_item.shape 
        global item_features_df
        global user_features_df
        item_features_df = rating_popular_item.pivot_table(
            index = self.itemTitle,
            #index = self.userId,
            columns = self.userId,
            #columns = self.itemTitle,
            values = self.rating).fillna(0)
        
        user_features_df= item_features_df.T
        user_features_df
        
        return user_features_df, item_features_df

    def knn_simialar_items(self, query_index, k :int):
        '''This function is to get the k nearest neighbors for items'''
       
        #convert the pivot table into a matrix
        item_features_df_matrix = csr_matrix(item_features_df.values)
        #create a model
        model_knn = NearestNeighbors(metric='cosine', algorithm= 'brute')
        
        #fit the model in the matrix
        model_knn.fit(item_features_df_matrix)

        #distances = what is the distance from the movie Im asking the recomendation for
        #indices = the index information of that particular movie
        distances, indices = model_knn.kneighbors(item_features_df.iloc[query_index,:].values.reshape(1,-1), n_neighbors = k+1)
       
        # I'm flatten the distances into 1Dimensional array
        # Print recommendations
        print('Item-based Recommendations for Item : \' {0} \': \n'.format(item_features_df.index[query_index+1]))
        for i in range(1, len(distances.flatten())):  # Start from 1 to skip the query item
            print('{0}: Item {1}, with distance of {2}:'.format(i,
                                                            item_features_df.index[indices.flatten()[i]],
                                                            distances.flatten()[i]))
        similarities = 1 - distances.flatten()
        return similarities, indices
    
    def knn_similar_users(self, query_user, k=5):
        '''This function is to get the k nearest neighbors for users'''
        #user_features_df = item_features_df.T  # Transpose for user-based
        user_features_df_matrix = csr_matrix(user_features_df.values)
        model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
        model_knn.fit(user_features_df_matrix)
        distances, indices = model_knn.kneighbors(user_features_df.loc[query_user, :].values.reshape(1, -1), n_neighbors=k + 1)

        print('User-based Recommendations for User: \'{0}\': \n'.format(query_user+1))
        for i in range(1, len(distances.flatten())):  # Start from 1 to skip the query user
            print('{0}: User {1}, with a distance of {2}:'.format(i,
                                                                user_features_df.index[indices.flatten()[i]],
                                                                distances.flatten()[i]))
        similarities = 1 - distances.flatten()
        return similarities, indices
    
    # predict the user base on the item title and the metric        
    def predict_userbased(self, userId, itemTitle, k ):
        prection  = 0
        user_loc = user_features_df.index.get_loc(userId)
        item_loc = item_features_df.columns.get_loc(itemTitle)
        similarities, indices = self.knn_similar_users(user_loc, k)
        mean_rating = user_features_df.iloc[user_loc, :].mean()
        sum_wt = np.sum(similarities) - 1
        product = 1
        wtd_sum = 0
        for i in range(0, len(indices.flatten())):
            if indices.flatten()[i] == user_loc:
                continue;
            else:
                ratings_diff = user_features_df.iloc[indices.flatten()[i], item_loc] - np.mean(user_features_df.iloc[indices.flatten()[i], :])
                product = ratings_diff * (similarities[i])
                wtd_sum = wtd_sum + product
        prection = int(round(mean_rating + (wtd_sum / sum_wt)))
        print('\nPredicted rating for user {0} -> item {1}: {2}'.format(userId, itemTitle, prection))
        return prection

    # predict the item base on the user id and the metric
    def predict_itembased(self, userId, itemTitle, k ):
        prection  = 0
        user_loc = user_features_df.index.get_loc(userId)
        item_loc = item_features_df.columns.get_loc(itemTitle)
        similarities, indices = self.knn_simialar_items(item_loc, k)
        sum_wt = np.sum(similarities) - 1
        product = 1
        wtd_sum = 0
        for i in range(0, len(indices.flatten())):
            if indices.flatten()[i] == item_loc:
                continue;
            else:
                product = user_features_df.iloc[user_loc, indices.flatten()[i]] * (similarities[i])
                wtd_sum = wtd_sum + product
        prection = int(round(wtd_sum / sum_wt))
        print('\nPredicted rating for user {0} -> item {1}: {2}'.format(userId, itemTitle, prection))
        return prection


In [3]:
cf = CollaborativeFiltering(
    #importing the data sets
    rating_df= '/home/bbruno/all_here/python course/thesis/knn/data/ratings.csv', 
    item_df= '/home/bbruno/all_here/python course/thesis/knn/data/movies.csv',
    #columns to use from the data sets 
    userId= 'userId', itemId= 'movieId', itemTitle= 'title', rating= 'rating')
cf.import_data()

(        userId  movieId  rating
 0            1        1     4.0
 1            1        3     4.0
 2            1        6     4.0
 3            1       47     5.0
 4            1       50     5.0
 ...        ...      ...     ...
 100831     610   166534     4.0
 100832     610   168248     5.0
 100833     610   168250     5.0
 100834     610   168252     5.0
 100835     610   170875     3.0
 
 [100836 rows x 3 columns],
       movieId                                      title
 0           1                           Toy Story (1995)
 1           2                             Jumanji (1995)
 2           3                    Grumpier Old Men (1995)
 3           4                   Waiting to Exhale (1995)
 4           5         Father of the Bride Part II (1995)
 ...       ...                                        ...
 9737   193581  Black Butler: Book of the Atlantic (2017)
 9738   193583               No Game No Life: Zero (2017)
 9739   193585                               Flint (

In [4]:
cf.clean_data()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)
...,...,...,...,...
100831,610,160341,2.5,Bloodmoon (1997)
100832,610,160527,4.5,Sympathy for the Underdog (1971)
100833,610,160836,3.0,Hazard (2005)
100834,610,163937,3.5,Blair Witch (2016)


In [5]:
#by default perItem = True and ofTable = False
cf.ratingCount()

#or
#perItem = True and ofTable = True
cf.ratingCount(ofTable= True)

,userId,movieId,rating,title,ratingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [6]:
cf.statistics()

count    9719.000000
mean       10.375141
std        22.406220
min         1.000000
25%         1.000000
50%         3.000000
75%         9.000000
max       329.000000
Name: ratingCount, dtype: float64

In [7]:
#.get_populars(by default popularity = 50)
cf.create_user_item_matrix()

(title   10 Things I Hate About You (1999)  12 Angry Men (1957)  \
 userId                                                           
 1                                     0.0                  0.0   
 2                                     0.0                  0.0   
 3                                     0.0                  0.0   
 4                                     0.0                  5.0   
 5                                     0.0                  0.0   
 ...                                   ...                  ...   
 606                                   0.0                  0.0   
 607                                   0.0                  0.0   
 608                                   0.0                  0.0   
 609                                   0.0                  0.0   
 610                                   0.0                  0.0   
 
 title   2001: A Space Odyssey (1968)  28 Days Later (2002)  300 (2007)  \
 userId                                             

In [8]:
cf.knn_simialar_items(query_index= 8, k= 4)

Item-based Recommendations for Item : ' Ace Ventura: When Nature Calls (1995) ': 

1: Item Dumb & Dumber (Dumb and Dumber) (1994), with distance of 0.3298064500857152:
2: Item Batman Forever (1995), with distance of 0.3795675902998943:
3: Item Die Hard: With a Vengeance (1995), with distance of 0.38145714569169953:
4: Item Ace Ventura: When Nature Calls (1995), with distance of 0.3950399602416229:


(array([1.        , 0.67019355, 0.62043241, 0.61854285, 0.60496004]),
 array([[  8, 136,  45, 127,   9]]))

In [9]:
cf.knn_similar_users(query_user=5, k=5)

User-based Recommendations for User: '6': 

1: User 470, with a distance of 0.37495563611500404:
2: User 565, with a distance of 0.39720942405438975:
3: User 235, with a distance of 0.4032123204915813:
4: User 229, with a distance of 0.406412874068513:
5: User 142, with a distance of 0.4167184916821013:


(array([1.        , 0.62504436, 0.60279058, 0.59678768, 0.59358713,
        0.58328151]),
 array([[  4, 466, 561, 232, 226, 140]]))

In [10]:
cf.predict_userbased(userId=5, itemTitle= 25, k= 5)

User-based Recommendations for User: '5': 

1: User 156, with a distance of 0.4817556052579327:
2: User 603, with a distance of 0.5109251479608923:
3: User 391, with a distance of 0.5180955409912863:
4: User 275, with a distance of 0.5186207686837994:
5: User 474, with a distance of 0.5405032228150742:

Predicted rating for user 5 -> item 25: -2


-2

In [11]:
cf.predict_itembased(userId=5, itemTitle= 25, k= 5)

Item-based Recommendations for Item : ' Animal House (1978) ': 

1: Item Zoolander (2001), with distance of 0.3642889634858275:
2: Item Superbad (2007), with distance of 0.39668781628511096:
3: Item Knocked Up (2007), with distance of 0.4307258776066052:
4: Item Wedding Crashers (2005), with distance of 0.43884375271335696:
5: Item Hangover, The (2009), with distance of 0.4470182027348636:

Predicted rating for user 5 -> item 25: 0


0